# deployment steps:
0) 
1) make sure that Dockerfile, requirements.txt and main.py is in repo
2) export vars to ~/.zshrc
!!!full paths to the credentials.json and token.json this is for local run
change here GOOGLE_CLOUD_PROJECT, GOOGLE_OAUTH_CLIENT_FILE, GOOGLE_OAUTH_TOKEN_FILE
```
export GOOGLE_GENAI_USE_VERTEXAI=True
export GOOGLE_CLOUD_PROJECT=personalplanner-476218       
export GOOGLE_CLOUD_LOCATION=us-central1
export GOOGLE_OAUTH_CLIENT_FILE="/Users/gman/Documents/hackathon/personalPlanner/.creds/credentials.json"  
export GOOGLE_OAUTH_TOKEN_FILE="/Users/gman/Documents/hackathon/personalPlanner/.creds/token.json"
export MODEL=gemini-2.5-flash
```
3) ```source ~/.zshrc```
NOTE to run the current verison of setup run with python main.py. the above exports are for local run only, not cloudrun!!!

4) deploy containers ( change project name in 2 places here)

```
gcloud run deploy project-planner-service \
  --source . \
  --region us-central1 \
  --project personalplanner-476218 \
  --allow-unauthenticated \
  --memory=1Gi \
  --set-env-vars=GOOGLE_GENAI_USE_VERTEXAI=True,GOOGLE_CLOUD_PROJECT=personalplanner-476218,GOOGLE_CLOUD_LOCATION=us-central1,MODEL=gemini-2.5-flash,GOOGLE_OAUTH_CLIENT_FILE=/app/.creds/credentials.json,GOOGLE_OAUTH_TOKEN_FILE=/app/.creds/token.json
```


5) run this ( Change project name )
```
gcloud run services describe project-planner-service \
  --region=us-central1 \
  --project=personalplanner-476218 \
  --format='value(spec.template.spec.serviceAccountName)'
```


it returns something like xxxxxxx-compute@developer.gserviceaccount.com
then run what's below with that thingy as your SA_EMAIL
```
SA_EMAIL="98380938461-compute@developer.gserviceaccount.com"      

gcloud projects add-iam-policy-binding personalplanner-476218 \
  --member="serviceAccount:${SA_EMAIL}" \
  --role="roles/aiplatform.user"

```
6) run this if session changed
```
gcloud run services update-traffic project-planner-service \
  --region us-central1 \
  --to-latest
```
7) idk anymore force cloud run to a single instance
```
gcloud run services update project-planner-service \
  --region us-central1 \
  --max-instances=1 \
  --min-instances=1

```

# Paste your service URL here 

In [1]:
import requests
import json
# Service [project-planner-service] revision [project-planner-service-00002-fhq] has been deployed and is serving 100 percent of traffic.
# Service URL: https://project-planner-service-98380938461.us-central1.run.app
SERVICE_URL = "https://project-planner-service-98380938461.us-central1.run.app"

# Helpers

In [2]:
APP_NAME = "orchestrator"  # from /list-apps
USER_ID = "grant"          # anything you like
SESSION_ID = "sess1"       # anything you like
def pretty_print(obj):
    print(json.dumps(obj, indent=2, ensure_ascii=False))


# 2) List available apps
def list_apps():
    url = f"{SERVICE_URL}/list-apps"
    resp = requests.get(url)
    resp.raise_for_status()
    apps = resp.json()
    print("Available apps:")
    pretty_print(apps)
    return apps


# 3) Create or update a session

def create_session(initial_state=None):
    if initial_state is None:
        initial_state = {}

    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.post(url, json=initial_state)

    print("Status:", resp.status_code)
    print("Headers:", resp.headers)
    print("Body:\n", resp.text)

    # Only raise if it's 4xx/5xx *after* we print
    resp.raise_for_status()

    try:
        session = resp.json()
        print("\nParsed session JSON:")
        pretty_print(session)
        return session
    except Exception as e:
        print("\nJSON parse error:", e)
        return None



# 4) Get session (optional, for debugging)
def get_session():
    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.get(url)
    resp.raise_for_status()
    session = resp.json()
    print("Current session:")
    pretty_print(session)
    return session


# 5) Run the orchestrator with a user message
def run_orchestrator(message_text: str):
    url = f"{SERVICE_URL}/run"
    payload = {
        "app_name": APP_NAME,
        "user_id": USER_ID,
        "session_id": SESSION_ID,
        "new_message": {
            "role": "user",
            "parts": [
                {"text": message_text}
            ]
        }
    }
    resp = requests.post(url, json=payload)
    print("Status:", resp.status_code)
    print("Body:\n", resp.text)
    resp.raise_for_status()
    events = resp.json()
    print("Events from orchestrator:")
    pretty_print(events)
    return events


# 6) Convenience helper: full flow in one call
def send_to_orchestrator(message_text: str, init_if_needed: bool = True):
    """
    - Optionally creates/updates a session first
    - Sends a message
    - Returns events
    """
    if init_if_needed:
        # you can attach any initial session state here if you want
        create_session(initial_state={})

    return run_orchestrator(message_text)


# 7) Delete session (if you want to reset state)
def delete_session():
    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.delete(url)
    if resp.status_code not in (200, 204):
        print("Delete returned:", resp.status_code, resp.text)
    else:
        print("Session deleted.")



In [3]:
apps = list_apps()

Available apps:
[
  "apollo_service",
  "calendar_service",
  "call_service",
  "gmail_service",
  "google_docs_service",
  "google_drive_service",
  "google_search_service",
  "google_sheets_service",
  "jobs_service",
  "matching_service",
  "orchestrator",
  "personalplanner.egg-info",
  "resume_customization",
  "services",
  "ui",
  "utils"
]


In [4]:
create_session(initial_state={"note": "session created from Jupyter"})


Status: 200
Headers: {'content-type': 'application/json', 'x-cloud-trace-context': 'cc27eaf10f9b9c1af07bb28f54d6db7b;o=1', 'date': 'Mon, 10 Nov 2025 12:17:04 GMT', 'server': 'Google Frontend', 'Content-Length': '146', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}
Body:
 {"id":"sess1","appName":"orchestrator","userId":"grant","state":{"note":"session created from Jupyter"},"events":[],"lastUpdateTime":1762777024.0}

Parsed session JSON:
{
  "id": "sess1",
  "appName": "orchestrator",
  "userId": "grant",
  "state": {
    "note": "session created from Jupyter"
  },
  "events": [],
  "lastUpdateTime": 1762777024.0
}


{'id': 'sess1',
 'appName': 'orchestrator',
 'userId': 'grant',
 'state': {'note': 'session created from Jupyter'},
 'events': [],
 'lastUpdateTime': 1762777024.0}

In [5]:
events = run_orchestrator(
    """Task: Customize my resume for this job.

Job title: Machine Learning Engineer  
Company: Stripe  
Job_row_id_in_sheet: 42  (if applicable)

Target skills for this job:
Machine Learning, Deep Learning, PyTorch, JAX, C++, Communication

What is missing:
The resume does not explicitly mention experience with JAX. While communication is implied
through project work and team collaboration, it is not explicitly highlighted as a distinct
skill, which could be beneficial for roles emphasizing this aspect.

Please update the LaTeX resume, rebuild the PDF, upload it to the configured Drive folder,
and return ONLY a single JSON object with:
status, job_title, company, drive_file_id, summary_of_changes.
""")


Status: 200
Body:
 [{"modelVersion":"gemini-2.5-flash","content":{"parts":[{"text":"I am sorry, but I cannot directly update a LaTeX resume, rebuild a PDF, or upload it to a specific Drive folder with the current tools. My capabilities do not include LaTeX compilation or PDF generation.","thoughtSignature":"CtMLAePx_15CdId_wD6bFvkrZbuVdEqaIEv2pv0F88YNEXIPX1C91aUzidtEouuIbkmmrLYpOa5kcjXYBSe3PprKBorx1jYt8VUkJnqyjWn5iqQMztL1KvENjsbUUBHyLytqGWmDlZOXKPzmxbUafPAmI5Veadqi5zJRLLrNn48f997Ic4_sxLw6b8CCLiZtUelX24lcuwi1MKIWaBQne3YhAxcHuyoPZzkdeL8hrVDZEncBaNb6ddp92YJJ16rEK7Qx1kqEtS0bxaOYer4Q9-mydAw_qbZ9EScMJAVWSxMiSliEAecT2bQtyvrbupW4SNFpzLwIq6uxKFYU1S5TMZZ_PB-sKOJkIZidKjSi73bkEon8_5Zwj_b8FqKLDhJPgQlxxbUWjOurPWP4Gi0kQhX9Ik9ZbYZgyvENjcA7Y0MGMTLcTQMwhOdWcSpMjlBpUC5c75UsQJjB-YBXcQHLNk-I52JbZ-iqwc41lJ3JYe2axjEaB85Nrwa9ETdgukZlvj6zehxPcPqFeTnbzqIKOVF6K2sPvBNyoV6f1roHSlAD38sXbkYnzmp0mltev0rK-d2OUtqzKEuqJQ93mMChjUpDIbAjnJI_IBZPFV7wdp2_IOMERrSnqAz7wNRbeWXT9d9mGftZxdqjNRBhmdRk__8oK6p2OqojnuXCDF37HWRo-HZyq9S

In [11]:
events

[{'modelVersion': 'gemini-2.5-flash',
  'content': {'parts': [{'functionCall': {'id': 'adk-a85fea39-c08b-4d5d-9e64-7cd2e8ca8eb5',
      'args': {'agent_name': 'google_drive_agent'},
      'name': 'transfer_to_agent'},
     'thoughtSignature': 'CokOAePx_16PW_QGcLRviPU13qI4DdxqdeO-dhzgLD6YaYtxfRYcOVyVpXPsAqSGkOzZJETcfK-MLmQAk7cUPs24f-gFKmLzqywWdezCSkEZMKDHZOJcboKbDbquuxYAZpkzOZDKom_KMw9ElD71ccslCG0fqzjpw60d8b2l4YdFkjqywXCKAFEx9UBE-7oxJ6pBBaZnpM2VFfGwtwG8YYs-4FfyzwjOhiQ8Bvp2VJrkPiipqkwRf0lmKwKtRGYjzct9OACG3W9ligXuksGA5glhcX_3d-Kj5kTB6MVa66DyhbvLCAvAcj9fU1C29lW_V2k7gq7Ah4gZWC5-4mF1lZ6DwJPtuJ-2SiWTYs-D5cj9IB8VxwK3K1lOEs7QaY2VpRKfX8pEaL2_OzCuquOhURk_Bv1d0EjoOfTXb2ncYpYQKUXOhA0b_1gye2n_xUfdi9B2enOj9BeXIt-h_b63ibQV4q4LxIFMruHHIOsXqHV3C60FmfrfZz16fgwTePN_Fz4rFwRUZcZMd0A3vAM7JkolIRSTEq3M7J_NIvQA4KXoSgS_2GJZZIJtx4B-tYFW6f5vzJS2ZUDqiqScKGakllw2nhmZhcDsxVJ07Z-ZtfANAjv0MddzoS072lzFoeK4e3g9_MMdkQQP-lEdkiMEpvIHeIvAes0D7woxLVjudThGH-NmFi6lqrFnQe_XqGmA44OgNHy1M-s7VBJQAHrLwTqnYdJsXBRjAkem4Jm97YN3CALksuoU

In [13]:
import re
last_with_text = None
for ev in reversed(events):
    if ev.get("modelVersion") and ev.get("content"):
        parts = ev["content"].get("parts", [])
        if parts and "text" in parts[0]:
            last_with_text = ev
            break

if last_with_text is None:
    raise ValueError("No model text response found in events")

raw_text = last_with_text["content"]["parts"][0]["text"]

# 2. Strip ```json ... ``` fences if they exist
match = re.search(r"```json\s*(.*?)\s*```", raw_text, re.DOTALL)
json_str = match.group(1) if match else raw_text

# 3. Parse the JSON array of resumes
resumes = json.loads(json_str)


In [16]:
resumes

[{'name': 'resume_Grant_Ovsepyan_DS_AI.pdf',
  'id': '1aqAiyY2SXM5DOjtMmDvj7baTkucMRHBt',
  'score': 90,
  'what_is_good': 'This resume demonstrates exceptional strength in Machine Learning, Deep Learning, and Natural Language Processing, with extensive experience in building and deploying complex models. It showcases robust skills in Big Data technologies, Cloud Computing (AWS, GCP, Azure), and MLOps practices, including CI/CD, Docker, and Kubernetes. The candidate also has strong programming skills in Python and SQL, and practical experience with ETL pipelines, model evaluation, and hyperparameter tuning.',
  'what_is_missing': 'While the resume is very strong, it could be enhanced by explicitly detailing experience in `Probability Theory` and `Experimental Design`, which are foundational for robust data science. More explicit examples of `Data Cleaning` and `Feature Engineering` within projects would also strengthen these areas. The resume does not mention `R` as a programming langu

In [3]:
import requests
import json
SERVICE_URL = "https://project-planner-service-98380938461.us-central1.run.app"
create_session(initial_state={"note": "session created from Jupyter"})

APP_NAME = "orchestrator"  # from /list-apps
USER_ID = "grant"          # anything you like
SESSION_ID = "sess1"       # anything you like
def pretty_print(obj):
    print(json.dumps(obj, indent=2, ensure_ascii=False))




run_orchestrator(
    """can you look for software engineer jobs posted for the past 7 days """
)

NameError: name 'create_session' is not defined

In [ ]:
run_orchestrator(
    """Tag jobs in the sheet that match 5 years experience in San Francisco, including remote jobs that qualify."""
)

# trash

In [10]:
from orchestrator_client import create_session, run_orchestrator, list_apps

list_apps()  # should show "orchestrator" in the list

create_session(initial_state={"note": "session created from REPL"})

run_orchestrator("can you look for software engineer jobs posted for the past 7 days")


Available apps:
[
  "apollo_service",
  "calendar_service",
  "gmail_service",
  "google_docs_service",
  "google_drive_service",
  "google_search_service",
  "google_sheets_service",
  "jobs_service",
  "matching_service",
  "orchestrator",
  "personalplanner.egg-info",
  "resume_customization",
  "services",
  "ui",
  "utils"
]

[create_session]
Status: 409
Body: {"detail":"Session already exists: sess1"} 
Session already exists; reusing it.

[run_orchestrator]
Status: 500
Body: Internal Server Error 


HTTPError: 500 Server Error: Internal Server Error for url: https://project-planner-service-98380938461.us-central1.run.app/run

In [ ]:



create_session(initial_state={"note": "session from REPL"})
run_orchestrator("can you look for software engineer jobs posted for the past 7 days")
